### Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [ ]:
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import ExactMatch

In [ ]:
data_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/data/nq_open/validation-00000-of-00001.parquet"

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet(data_file_path, engine='pyarrow')

In [ ]:
df.head()

,question,answer
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]"
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]"
2,how many seasons of the bastard executioner ar...,"[one, one season]"
3,when did the eagles win last super bowl,[2017]
4,who won last year's ncaa women's basketball,[South Carolina]


### Load LLM

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 26.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.57-cp310-cp310-manylinux_2_35_x86_64.whl size=26393815 sha256=b22397c942a6b59425372fd7f842151b36534921ff3ff12f2061a2aa4bc48092
  Stored in directory: /root/.cache/pip/wheels/7e/c0/00/e98d6e198f941c623da37b3f674354cbdccfcfb2cb9cf1133d
Successfully built llama-cpp-python


In [ ]:
%pip install llama-index-llms-llama-cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00


In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

n_gpu_layers = 40 # Change this value based on your model and your GPU VRAM pool.
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": n_gpu_layers},
    #verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_co

### Inference

In [ ]:
!pip install tqdm

In [ ]:
PROMPT_TEMPLATE = """
[INST] <<SYS>>
This is a Open Domain Question Answering Task. You are given a question and you need to generate
the short answer based on the question. Below are a few examples:
Question: where did they film hot tub time machine
Answer: Fernie Alpine Resort
Question: who has the right of way in international waters
Answer: Neither vessel
Question: who does annie work for attack on titan
Answer: Marley
Question: when was the immigration reform and control act passed
Answer: November 6, 1986
[ENDSYS]
<</SYS>>
Question: {prompt}[/INST]
"""

In [ ]:
def generate_answer(question, index):
    # Format the PROMPT_TEMPLATE with the actual question
    formatted_prompt = PROMPT_TEMPLATE.format(prompt=question)
    try:
        response = llm.complete(formatted_prompt)
        answer = response.text
        print(f"Index: {index}, Question: '{question}', Generated Answer: '{answer}'")
        return answer
    except Exception as e:
        print(f"Error generating answer for question '{question}' at index {index}: {e}")
        return None

In [ ]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_llama2.csv"

In [ ]:
try:
    # Try to load the DataFrame from the CSV file
    df_eval = pd.read_csv(eval_file_path)
    print("Data loaded from file.")
except FileNotFoundError:
    # If the file doesn't exist, copy df into df_eval
    df_eval = df.copy()
    if 'predictions' not in df_eval.columns:
      df_eval['predictions'] = pd.NA  # or use None or np.nan, depending on your preference
    print("File not found. Copied df into df_eval.")

File not found. Copied df into df_eval.


In [ ]:
from tqdm import tqdm

# Initialize a counter for tracking when to save the DataFrame
save_counter = 0

for index, row in tqdm(df_eval.iterrows(), total=df_eval.shape[0], desc="Generating Answers"):
    # Check if 'predictions' is empty for the current row
    if pd.isna(row['predictions']) or row['predictions'] == '':
        # Generate the answer only if 'predictions' is empty
        df_eval.at[index, 'predictions'] = generate_answer(row['question'], index)

        # Increment the save counter
        save_counter += 1

        # Save the DataFrame back to the file every 10 rows
        if save_counter % 10 == 0:
            df_eval.to_csv(eval_file_path, index=False)
            print(f"Progress saved at index {index}")

# Save any remaining progress at the end of the loop
if save_counter % 10 != 0:
    df_eval.to_csv(eval_file_path, index=False)
    print("Final progress saved.")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df_eval.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",Answer: The last time anyone was on the moon w...
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",Answer: Bobby Scott and Bob Russell.
2,how many seasons of the bastard executioner ar...,"[one, one season]","Answer: There is only one season of ""The Basta..."
3,when did the eagles win last super bowl,[2017],Answer: The Eagles won their last Super Bowl i...
4,who won last year's ncaa women's basketball,[South Carolina],Answer: Baylor Lady Bears


## Save evaluation data

In [ ]:
df_eval.to_csv(eval_file_path, index=False)

## Evaluation metrics

In [1]:
!pip install pandas pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_llama2.csv"

In [4]:
df = pd.read_csv(eval_file_path)

In [5]:
df.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,['14 December 1972 UTC' 'December 1972'],Answer: The last time anyone was on the moon w...
1,who wrote he ain't heavy he's my brother lyrics,['Bobby Scott' 'Bob Russell'],Answer: Bobby Scott and Bob Russell.
2,how many seasons of the bastard executioner ar...,['one' 'one season'],"Answer: There is only one season of ""The Basta..."
3,when did the eagles win last super bowl,['2017'],Answer: The Eagles won their last Super Bowl i...
4,who won last year's ncaa women's basketball,['South Carolina'],Answer: Baylor Lady Bears


In [6]:
print(df.iloc[1]['answer'])

['Bobby Scott' 'Bob Russell']


In [7]:
import ast
import string
import re
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import normalize_answer

def string_to_list(string):
    try:
        # Convert the string to a list
        matches = re.findall(r"'(.*?)'", string)
        return matches
    except Exception as e:
        # In case of an error, print the error and return the original string
        print(f"Error converting string to list: {e}")
        return string


def preprocess(text: str, answers: list) -> str:
    text_after_answer = re.sub(r'^Answer: ', '', text)

    exclude = set(string.punctuation)
    exclude.add("’")
    exclude.add("\n")

    # Remove excluded characters
    cleaned_text = normalize_answer("".join(ch for ch in text_after_answer if ch not in exclude))

    # Replace text with answer item if it matches a substring in the cleaned text
    for answer_item in answers:
        if normalize_answer(answer_item) in cleaned_text:
            return answer_item  # Return the answer item that matches the substring

    return cleaned_text

In [8]:
df['answer'] = df['answer'].apply(string_to_list)
df['predictions'] = df['predictions'].str.replace("^Answer: ", "", regex=True)
df['predictions_em'] = df.apply(lambda row: preprocess(row['predictions'], row['answer']), axis=1)

In [9]:
df.head()

,question,answer,predictions,predictions_em
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",The last time anyone was on the moon was durin...,December 1972
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",Bobby Scott and Bob Russell.,Bobby Scott
2,how many seasons of the bastard executioner ar...,"[one, one season]","There is only one season of ""The Bastard Execu...",one
3,when did the eagles win last super bowl,[2017],The Eagles won their last Super Bowl in 2018.,eagles won their last super bowl in 2018
4,who won last year's ncaa women's basketball,[South Carolina],Baylor Lady Bears,baylor lady bears


### Calculate Scores

In [10]:
import datasets
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import exact_match_score, metric_max_over_ground_truths, f1_score

df['em'] = df.apply(lambda row: metric_max_over_ground_truths(exact_match_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)
df['f1'] = df.apply(lambda row: metric_max_over_ground_truths(f1_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)

In [11]:
!pip install -q rouge_score
!pip install -q evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [12]:
import evaluate

df['predictions_em'] = df['predictions_em'].astype(str)
df['answer'] = df['answer'].astype(str)

# Filter out rows where either 'predictions' or 'answer' is empty or NaN
filtered_df = df[(df['predictions_em'].notna() & df['predictions_em'].str.strip().astype(bool)) &
                 (df['answer'].notna() & df['answer'].str.strip().astype(bool))]
rouge = evaluate.load('rouge')
predictions = filtered_df['predictions_em'].tolist()
references = filtered_df['answer'].tolist()
rouge_results = rouge.compute(predictions=predictions,
                        references=references)
print(rouge_results)

{'rouge1': 0.3039855839107419, 'rouge2': 0.1489635662867857, 'rougeL': 0.3022293294707674, 'rougeLsum': 0.3022555400966293}


### Prove of correctness

#### Top 10 rows from data frame

In [13]:
df.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",The last time anyone was on the moon was durin...,December 1972,True,1.0
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",Bobby Scott and Bob Russell.,Bobby Scott,True,1.0
2,how many seasons of the bastard executioner ar...,"['one', 'one season']","There is only one season of ""The Bastard Execu...",one,True,1.0
3,when did the eagles win last super bowl,['2017'],The Eagles won their last Super Bowl in 2018.,eagles won their last super bowl in 2018,False,0.0
4,who won last year's ncaa women's basketball,['South Carolina'],Baylor Lady Bears,baylor lady bears,False,0.0
5,when did the isle of wight become an island,['During the last Ice Age'],The Isle of Wight has been an island for most ...,During the last Ice Age,True,1.0
6,love yourself by justin bieber is about who,['Rihanna'],"Justin Bieber's song ""Love Yourself"" is about ...",justin biebers song love yourself is about lov...,False,0.0
7,who was the ruler of england in 1616,['James I'],King James I of England (1603-1625),James I,True,1.0
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],The Hot Coffee mod in San Andreas is a popular...,hot coffee mod in san andreas is popular mod t...,False,0.0
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],54 Mbps (megabits per second),54 mbps megabits per second,False,0.0


#### Top 10 rows from data frame that exactly match one of the reference answer

In [14]:
df_em_true = df[df['em'] == True]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",The last time anyone was on the moon was durin...,December 1972,True,1.0
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",Bobby Scott and Bob Russell.,Bobby Scott,True,1.0
2,how many seasons of the bastard executioner ar...,"['one', 'one season']","There is only one season of ""The Bastard Execu...",one,True,1.0
5,when did the isle of wight become an island,['During the last Ice Age'],The Isle of Wight has been an island for most ...,During the last Ice Age,True,1.0
7,who was the ruler of england in 1616,['James I'],King James I of England (1603-1625),James I,True,1.0
10,which state is located in the centre of india,"['Chhattisgarh', 'Madhya Pradesh']",Madhya Pradesh,Madhya Pradesh,True,1.0
14,who does the voice of nala in the lion king,"['Niketa Calame', 'Sally Dworsky', 'Moira Kell...",Moira Kelly,Moira Kelly,True,1.0
17,who is under the mask of darth vader,['Anakin Skywalker'],Anakin Skywalker,Anakin Skywalker,True,1.0
18,who had a baby at 100 in the bible,"['Sarah', 'Abraham']","Sarah, who had a son named Isaac when she was ...",Sarah,True,1.0
19,what age do you need to be to buy a bb gun,['18'],"In most states, you must be at least 18 years ...",18,True,1.0


#### Top 10 rows from data frame that doesn't match match any of the reference answer

In [15]:
df_em_true = df[df['em'] == False]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
3,when did the eagles win last super bowl,['2017'],The Eagles won their last Super Bowl in 2018.,eagles won their last super bowl in 2018,False,0.000000
4,who won last year's ncaa women's basketball,['South Carolina'],Baylor Lady Bears,baylor lady bears,False,0.000000
6,love yourself by justin bieber is about who,['Rihanna'],"Justin Bieber's song ""Love Yourself"" is about ...",justin biebers song love yourself is about lov...,False,0.000000
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],The Hot Coffee mod in San Andreas is a popular...,hot coffee mod in san andreas is popular mod t...,False,0.000000
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],54 Mbps (megabits per second),54 mbps megabits per second,False,0.000000
11,who sang i ran all the way home,['The Impalas'],The Beatles,beatles,False,0.000000
12,where did the last name wallace come from,['a Scottish surname'],"The last name ""Wallace"" originated in Scotland...",last name wallace originated in scotland and i...,False,0.037736
13,who was the actor that played ben stone on law...,['Michael Moriarty'],Sam Waterston,sam waterston,False,0.000000
15,who plays gram on the young and the restless,['Max Shippee'],Jess Walton,jess walton,False,0.000000
16,what is the oath that new citizens take,['United States Oath of Allegiance'],The Oath of Allegiance.,oath of allegiance,False,0.750000


#### Overall Exact Match & F1 & Rouge scores

In [16]:
overall_em = sum(df['em']) / len(df['em'])
overall_f1 = sum(df['f1']) / len(df['f1'])

# Print the overall Exact Match score, F1 and Rouge score
print(f"Overall Exact Match Score: {overall_em * 100:.2f}")
print(f"Overall F1 Score: {overall_f1 * 100:.2f}")
print(f"Rouge1: {rouge_results['rouge1'] * 100:.2f}")
print(f"Rouge2: {rouge_results['rouge2'] * 100:.2f}")
print(f"RougeL: {rouge_results['rougeL'] * 100:.2f}")
print(f"RougeLSum: {rouge_results['rougeLsum'] * 100:.2f}")

Overall Exact Match Score: 35.96
Overall F1 Score: 40.16
Rouge1: 30.40
Rouge2: 14.90
RougeL: 30.22
RougeLSum: 30.23


#### Save with metrics

In [17]:
eval_with_metrics_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_llama2_metrics.csv"

In [18]:
df.to_csv(eval_with_metrics_file_path, index=False)